# Group Members: Adrian, Aryan, Hon Joo

For this project, we have decided to go with a dataset containing 2018 flight data. While 2018 isn't the most up to date data,  data from parts of 2019 and 2020 onwards aren't reliable due to the Covid-19 pandemic. 


In [ ]:
# Basic Libraries
!pip install missingno
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot
import matplotlib.cm as cm
from sklearn.cluster import DBSCAN
from sklearn import metrics
import time
sb.set() # set the default Seaborn style for graphics
import missingno as msno 


In [ ]:
flightdata = pd.read_csv("2018.csv")
flightdata_df = pd.DataFrame(flightdata)
flightdata_df.head()

In [ ]:
flightdata_df.info(verbose = True, show_counts = True)

---

We have 7213446  flights in the dataset, which is a hefty amount. We noticed that some of the columns have many missing values. 

For example, for flights with weather delay (WEATHER_DELAY), there's only 1352710 data points. Although, it is still a significant number.

We can use missingno library to visually represent the missing values in each column.

---

In [ ]:
msno.matrix(flightdata)

---

We'll check the columns of CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY against ARR_DELAY and DEP_DELAY to see if there any relation

---

In [ ]:
datalist = ['CARRIER_DELAY','WEATHER_DELAY','NAS_DELAY','SECURITY_DELAY','LATE_AIRCRAFT_DELAY','ARR_DELAY','DEP_DELAY']

# simple function to print out n rows of particular column(s) of a dataframe
# requires explicit list of individual column names

def printcolumns(dataframe, columnlist, n):
    return dataframe[columnlist].head(n)



In [ ]:
printcolumns(flightdata_df, datalist, 20)

---

We picked out 20 data to take a quick look. Naively, it seems like the sum of all the delays equates to ARR_DELAY. This means that the arrival delay is split into the different types of delays that accumulate. 

We also noticed that some entries in the ARR_DELAY and DEP_DELAY columns are negative. This means that the flights were either on time or early. We want to treat all early flights and on time flights as the same.

We will create 2 new columns, ARR_DELAY_NEW and DEP_DELAY_NEW, where any negative value is converted to 0.

---

In [ ]:
# function to convert negative values in a particular column to zeroes and apply it into a new column

def negativeToZero(dataframe, newcolumn, inputcolumn):
    dataframe[newcolumn] = dataframe[inputcolumn].apply(lambda x: max(x, 0))

In [ ]:
negativeToZero(flightdata_df,'ARR_DELAY_NEW','ARR_DELAY')
negativeToZero(flightdata_df,'DEP_DELAY_NEW','DEP_DELAY')


---

We have successfully converted the negative values in ARR_DELAY and DEP_DELAY to zero and created two new columns containing the cleaned data. From the table below, we can see that negative values have all been converted to zeroes.

---

In [ ]:
negtozerolist = ['ARR_DELAY','ARR_DELAY_NEW','DEP_DELAY','DEP_DELAY_NEW']
printcolumns(flightdata_df, negtozerolist, 20)

---

Using .info(), we can see that we have two new columns.

---

In [ ]:
flightdata_df.info()

---

We see that the new columns are placed at the 28th and 29th index. 

To make the dataset cleaner, we should place them after ARR_DELAY and DEP_DELAY.

---

In [ ]:
# simple function to shift a particular column to n-th index of the dataframe

def columnshifter(dataframe, n, columnname):
    temp_column = dataframe.pop(columnname)
    dataframe.insert(n, columnname, temp_column)

In [ ]:
columnshifter(flightdata_df, 15, 'ARR_DELAY_NEW')
columnshifter(flightdata_df, 8, 'DEP_DELAY_NEW')

In [ ]:
flightdata_df.info(verbose = True, show_counts = True)

In [ ]:
msno.matrix(flightdata_df)

---

After shifting the columns, we can see that ARR_DELAY_NEW and DEP_DELAY_NEW are placed after ARR_DELAY and DEP_DELAY respectively. 

---

To clean the data further, we chose certain criteria to drop rows by.

-no data in arrival delay column

-no data in departure delay column

---

In [ ]:
flightdata_df = flightdata_df.dropna(subset = ['ARR_DELAY']).reset_index(drop = True)
flightdata_df = flightdata_df.dropna(subset = ['DEP_DELAY']).reset_index(drop = True)

In [ ]:
flightdata_df.info(verbose = True, show_counts = True)

In [ ]:
msno.matrix(flightdata_df)

---

Now, we have trimmed the dataset slightly to just 7071818 rows. 

We noticed that the number of CANCELLATION_CODE entries have become zero. It seems that flights without any delay data whatsoever are cancelled flights.

We shall explore the columns to check out it's mean/median data to look for outliers. The pertinent columns would be delay data, so we shall look at those. 

---

In [ ]:
flightdata_df[['ARR_TIME','ARR_DELAY_NEW','DEP_DELAY_NEW','DEP_DELAY','CARRIER_DELAY','WEATHER_DELAY','NAS_DELAY','SECURITY_DELAY','LATE_AIRCRAFT_DELAY']].describe()

---

As we can see, for the row 'max', some of the delays(min) are in the thousands! They would definitely be classed as outliers.

We will be removing them during EDA.

---

# Encoding categorical data

We see that the flight carrier names are abbreviated. We can give names to these abbreviations for easier visualisation.

---

In [ ]:
# simple func that displays all the unique values of a particular column
# returns a list of all unique values of said column

def displayUnique(dataframe, column):
    outputlist = dataframe[column].unique().tolist()
    return outputlist

In [ ]:
carrierlist = displayUnique(flightdata_df, 'OP_CARRIER')
carrierlist

For readability and reference purposes, we will maintain a dictionary of the full names of each air carrier.

Let's also convert the abbrievations of each carrier into their full names, so that the final reference dictionary contains the full name of each carrier mapped to an unique index. 

---

In [ ]:
carrierdict = {
    'UA':'United Airlines',
    'AS':'Alaska Airlines',
    '9E':'Endeavor Air',
    'B6':'JetBlue Airways',
    'EV':'ExpressJet',
    'F9':'Frontier Airlines',
    'G4':'Allegiant Air',
    'HA':'Hawaiian Airlines',
    'MQ':'Envoy Air',
    'NK':'Spirit Airlines',
    'OH':'PSA Airlines',
    'OO':'SkyWest Airlines',
    'VX':'Virgin America',
    'WN':'Southwest Airlines',
    'YV':'Mesa Airline',
    'YX':'Republic Airways',
    'AA':'American Airlines',
    'DL':'Delta Airlines'
}

In [ ]:
# simple function to replace given data in a column to another based on 
# an input dictionary

def replaceEntries(dataframe, column, inputdict):
    dataframe[column].replace(inputdict,inplace=True)

In [ ]:
replaceEntries(flightdata_df,'OP_CARRIER', carrierdict)

In [ ]:
carrierlistFULL = displayUnique(flightdata_df, 'OP_CARRIER')
carrierlistFULL

---

Now that we have our list of full names of the air carriers, now lets generate a dictionary by mapping each one to a unique value.

---

In [ ]:
# simple function to generate a dictionary given an input list. 
# this function maps each entry in the list into a unique value index
# this is because the dataframe columns contains the keys, and we want to replace them with values containing the indexes
# if you want the indexes to be on the keys instead then just modify the function slightly

def generateDict(inputlist):
    values = range(len(inputlist))
    outputdict = dict(zip(inputlist, values))
    return outputdict

In [ ]:
carrierindex = generateDict(carrierlistFULL)
carrierindex

---

With the dictionary generated, now lets replace the air carrier entries in the OP_CARRIER column with the indexes

---

In [ ]:
replaceEntries(flightdata_df, 'OP_CARRIER', carrierindex)

In [ ]:
carrierlist_replaced = displayUnique(flightdata_df, 'OP_CARRIER')
carrierlist_replaced

We can see that the OP_CARRIER column has been replaced by indexes. We can refer to carrierindex dictionary in the future to check which air carrier an index belongs to.


Next, we want to simplify the destination and source airport columns as well. We can do this by assigning an index to each invidiual unique column. 

We can use the same methods that we used on OP_CARRIER on ORIGIN and DEST columns.


First, lets explore these columns.

---

In [ ]:
destlist = displayUnique(flightdata_df, 'DEST')
# destlist

# uncomment the above to print the list if you want, but its pretty large (300+ unique vals)


In [ ]:
originlist = displayUnique(flightdata_df, 'ORIGIN')
# originlist

# uncomment the above to print the list if you want, but its pretty large (300+ unique vals)


In [ ]:
destDict = generateDict(destlist)
originDict = generateDict(originlist)

In [ ]:
# destDict

In [ ]:
# originDict

---

After inspecting both dictionaries, we noticed that both columns are mapped differently. For example, for destDict, 'DEN' is mapped to 0 but for originDict, 'EWR' is mapped to 0.

This is because each column has the same amount of unique values, but do not have the exact same entries per row, as origin and destination airports are different.

So we shall drop one and only use one as the reference. We shall use destDict for future reference.

To avoid confusion, lets rename destDict to airportDict.

---

In [ ]:
airportDict = destDict
airportDict

In [ ]:
replaceEntries(flightdata_df, 'DEST', airportDict)
replaceEntries(flightdata_df, 'ORIGIN', airportDict)

In [ ]:
# destlist_replaced

In [ ]:
# originlist_replaced

In [ ]:
# sanitycheck
# crosscheck with csv file shows that the origin and dest mapping works
flightdata_df.head(50)

In [ ]:
flightdata_df.to_csv (r'INPUT PATH HERE', index = False, header=True)

# note: change the path name inside the quotation marks to the address where you want the CSV to be exported to.
# make sure the imported file ends with (.csv). You can also use .txt if you want.
# after running this code snippet the CSV will be downloaded into the path address